In [1]:
import h5py
import numpy as np

In [2]:
f5 = h5py.File("inpatient_readmission_analysis.hdf5", "r")

In [ ]:
independent_array = f5["/independent/core_array"][...]
independent_array.shape

In [4]:
independent_labels = f5["/independent/column_annotations"][...]
independent_labels[0: 10]

array(['gender_concept_name|FEMALE', 'Length of stay in days',
       'age_at_visit_start_in_years_int',
       'Past history of 30-day inpatient readmissions',
       'No matching concept|0|C',
       'Gingival and periodontal disease|132344|C',
       'Staphylococcal scalded skin syndrome|132392|C',
       'Pressure ulcer stage 1|132393|C',
       'Post-laminectomy syndrome|132412|C',
       'Chronic osteomyelitis of hand|132414|C'],
      dtype='|S128')

In [5]:
dependent_array = f5["/dependent/core_array"][...]
dependent_array = np.ndarray.flatten(dependent_array)
dependent_array.shape

(66700L,)

In [6]:
np.sum(dependent_array)

6421.0

In [7]:
dependent_labels = f5["/dependent/column_annotations"][...]
dependent_labels

array(['30-day inpatient readmission'],
      dtype='|S128')

In [8]:
import sklearn
sklearn.__version__

'0.19.0'

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(independent_array, dependent_array, test_size=0.3)

In [11]:
X_test

array([[71, 11,  1, ...,  0,  0,  0],
       [52,  7,  1, ...,  0,  0,  0],
       [79,  9,  0, ...,  0,  0,  0],
       ..., 
       [48,  5,  1, ...,  0,  0,  0],
       [68,  2,  1, ...,  0,  0,  0],
       [77,  3,  1, ...,  0,  0,  0]])

In [12]:
from sklearn.pipeline import Pipeline

In [13]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier

In [14]:
variance_thresh = VarianceThreshold()
select_k = SelectKBest(k=100)
random_forest = RandomForestClassifier()

pipeline_steps = [("remove_zero_variance", variance_thresh), 
                  ("select_features", select_k), 
                  ("random_forest", random_forest)]

In [15]:
pipeline_obj = Pipeline(pipeline_steps)

In [16]:
pipeline_obj.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('remove_zero_variance', VarianceThreshold(threshold=0.0)), ('select_features', SelectKBest(k=100, score_func=<function f_classif at 0x000000000734B908>)), ('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='a...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [17]:
Y_predict = pipeline_obj.predict(X_test)
Y_predict[0: 100]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

In [18]:
from sklearn.metrics import classification_report

In [19]:
report = classification_report(Y_predict, Y_test)

In [20]:
print(report)

             precision    recall  f1-score   support

        0.0       1.00      0.90      0.95     19955
        1.0       0.00      0.11      0.01        55

avg / total       0.99      0.90      0.95     20010



In [21]:
from sklearn.metrics import roc_curve

In [22]:
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
Y_test_p = pipeline_obj.predict_proba(X_test)

In [ ]:
fpr, tpr, _ = roc_curve(Y_test, Y_test_p[:,1])

In [ ]:
%matplotlib inline

In [ ]:
sb.set_style("darkgrid")
plt.plot(fpr, tpr)